In [5]:
# Quantum discrete Gaussian over x = [-1, 0, 1]
# Requires: qiskit

import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector


def f(u: float, T: float) -> np.ndarray:
    """
    Return a length-3 numpy array of probabilities [p(-1), p(0), p(1)]
    where the probabilities follow a discrete Gaussian with mean `u`
    and variance `T` over x = [-1, 0, 1].

    The function prepares a 2-qubit quantum state (using Initialize) whose
    measurement probabilities on |00>,|01>,|10> map to x = -1,0,1.

    Args:
        u: mean (float)
        T: variance (float), must be > 0

    Returns:
        numpy.ndarray of shape (3,) with probabilities in order [-1, 0, 1]
    """
    allowed = np.array([-1.0, 0.0, 1.0])

    if T <= 0:
        raise ValueError("variance T must be > 0")

    # classical discrete Gaussian (unnormalized)
    exps = np.exp(- (allowed - u)**2 / (2.0 * T))
    probs = exps / np.sum(exps)

    # prepare amplitudes for basis states |00>,|01>,|10>, leaving |11> zero
    amps = np.sqrt(probs)
    statevec = np.zeros(4, dtype=complex)
    statevec[0:3] = amps  # |00>->-1, |01>->0, |10>->1
    statevec = statevec / np.linalg.norm(statevec)

    qc = QuantumCircuit(2)
    qc.initialize(statevec, qc.qubits)

    sv = Statevector.from_instruction(qc)
    measured_probs = np.abs(sv.data)**2

    return np.array([measured_probs[0], measured_probs[1], measured_probs[2]])


# Example usage
if __name__ == "__main__":
    probs = f(0.2, 0.5)
    print("Probabilities [p(-1), p(0), p(1)]:", probs)


Probabilities [p(-1), p(0), p(1)]: [0.13734866 0.55697628 0.30567506]


In [20]:
#Test with T=1/3, u=(-0.4.0.4)*sqrt(1/3)
for u in np.linspace(-0.4, 0.4, 5) * np.sqrt(1/3):
    print(f"u={u:.3f}, probs={f(u, 1/3)}")
    #Recovered u,T from moments of probs:
    p = f(u, 1/3)
    mean = p[2] - p[0]
    var = p[0] + p[2] - mean**2
    print(f"  recovered mean={mean:.3f}, var={var:.3f}")  

u=-0.231, probs=[0.28639012 0.64196553 0.07164435]
  recovered mean=-0.215, var=0.312
u=-0.115, probs=[0.21414587 0.67874619 0.10710794]
  recovered mean=-0.107, var=0.310
u=0.000, probs=[0.15428077 0.69143845 0.15428077]
  recovered mean=0.000, var=0.309
u=0.115, probs=[0.10710794 0.67874619 0.21414587]
  recovered mean=0.107, var=0.310
u=0.231, probs=[0.07164435 0.64196553 0.28639012]
  recovered mean=0.215, var=0.312


In [7]:
# New function: sampling-based quantum Gaussian generator (sampling only, no analytic Gaussian formula)
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector


def quantum_gaussian_sampler(u, T, shots=2000, N=None, positions=None, seed=None):
    """
    Sampling-based approximate Gaussian generator using a product state of biased qubits.

    This function does NOT compute Gaussian probabilities analytically. Instead it:
      - prepares N independent biased qubits (each qubit maps {0,1} -> {-1,+1}),
      - samples the product-state measurement distribution `shots` times,
      - computes S = sum_i X_i for each shot (X_i in {-1,+1}),
      - applies affine rescaling Y = a*(S - S_mean) + u so that E[Y]=u and Var[Y]=T
        (matching theoretical S mean/var), producing samples whose empirical
        distribution approximates a Gaussian by the CLT.

    Args:
        u (float): desired mean
        T (float): desired variance (must be > 0)
        shots (int): number of measurement samples to draw
        N (int or None): number of qubits to prepare; if None, chosen automatically
        positions (array-like or None): optional discrete positions to bin samples onto
        seed (int or None): RNG seed for reproducibility

    Returns:
        dict: {
            'samples': array of float samples (length=shots),
            'raw_sums': integer sums S before rescaling,
            'hist': (values, probs) empirical histogram (binned if positions provided),
            'qc': the prepared QuantumCircuit,
            'params': {N, p, S_mean, S_var}
        }
    """
    if T <= 0:
        raise ValueError('variance T must be > 0')

    # choose N modestly (larger N improves CLT; keep small to avoid large statevectors)
    if N is None:
        N = max(2, min(20, int(np.ceil(8 * float(T)))))
    else:
        N = int(max(1, N))

    # choose per-qubit bias p so that E[S] = N*(2p-1) = u => p = (u/N + 1)/2
    p = (float(u) / float(N) + 1.0) / 2.0
    p = float(np.clip(p, 1e-12, 1.0 - 1e-12))

    # prepare N-qubit product state: each qubit sqrt(1-p)|0> + sqrt(p)|1> via Ry
    theta = 2.0 * np.arcsin(np.sqrt(p))
    qc = QuantumCircuit(N)
    for q in range(N):
        qc.ry(theta, q)

    # get exact statevector probabilities and sample from them using numpy RNG
    sv = Statevector.from_instruction(qc)
    probs = np.abs(sv.data) ** 2

    rng = np.random.default_rng(seed)
    outcomes = rng.choice(len(probs), size=shots, p=probs)

    # convert basis index to sum S in {-N,..,N}
    raw_sums = np.empty(shots, dtype=int)
    for i, idx in enumerate(outcomes):
        S = 0
        for j in range(N):
            bit = (idx >> j) & 1
            S += (2 * bit - 1)
        raw_sums[i] = S

    # theoretical mean and variance of S
    S_mean = N * (2.0 * p - 1.0)
    S_var = 4.0 * N * p * (1.0 - p)

    # compute scaling to match requested variance T
    if S_var <= 0.0:
        a = 0.0
    else:
        a = np.sqrt(float(T) / float(S_var))

    samples = a * (raw_sums - S_mean) + float(u)

    # optional binning to user-provided discrete positions
    if positions is not None:
        positions = np.asarray(positions, dtype=float)
        idxs = np.argmin(np.abs(samples[:, None] - positions[None, :]), axis=1)
        counts = np.bincount(idxs, minlength=len(positions)).astype(float) / float(shots)
        hist = (positions, counts)
    else:
        vals, counts = np.unique(samples, return_counts=True)
        hist = (vals, counts.astype(float) / float(shots))

    return {
        'samples': samples,
        'raw_sums': raw_sums,
        'hist': hist,
        'qc': qc,
        'params': {'N': N, 'p': p, 'S_mean': S_mean, 'S_var': S_var}
    }

In [8]:
#Test quantum_gaussian_sampler with T=1/3, u=(-0.4, 0.4)*sqrt(1/3)
for u in np.linspace(-0.4, 0.4, 5) * np.sqrt(1/3):
    result = quantum_gaussian_sampler(u, 1/3, shots=5000, seed=42)
    hist_vals, hist_probs = result['hist']
    print(f"u={u:.3f}, hist vals={hist_vals}, probs={hist_probs}")
    #Recovered u,T from moments of samples:
    samples = result['samples']
    mean = np.mean(samples)
    var = np.var(samples)
    print(f"  recovered mean={mean:.3f}, var={var:.3f}")



u=-0.231, hist vals=[-1.15670716 -0.48805636  0.18059443  0.84924523], probs=[0.1596 0.398  0.3448 0.0976]
  recovered mean=-0.234, var=0.335
u=-0.115, hist vals=[-1.07769306 -0.41053201  0.25662903  0.92379007], probs=[0.141  0.3882 0.3636 0.1072]
  recovered mean=-0.119, var=0.330
u=0.000, hist vals=[-1.         -0.33333333  0.33333333  1.        ], probs=[0.1244 0.3814 0.372  0.1222]
  recovered mean=-0.005, var=0.330
u=0.115, hist vals=[-0.92379007 -0.25662903  0.41053201  1.07769306], probs=[0.111  0.3704 0.3824 0.1362]
  recovered mean=0.106, var=0.330
u=0.231, hist vals=[-0.84924523 -0.18059443  0.48805636  1.15670716], probs=[0.0966 0.3606 0.3876 0.1552]
  recovered mean=0.222, var=0.332
